In [ ]:
import cv2
import numpy as np
import dlib
from time import sleep
import sys


predictor_path ="shape_predictor_68_face_landmarks.dat"

scale_Factor=1
feather_Amount =11

Face_points =list(range(17,68))
mouth_points =list(range(48,61))
right_brow_points =list(range(17,22))
left_brow_points= list(range(22,27))
right_eye=list(range(36,42))
left_eye=list(range(42,48))
nose_points =list(range(27,35))
jaw_points=list(range(0,17))

Align_points=(mouth_points+right_brow_points+right_eye+left_brow_points+left_eye+nose_points)

overlay_points =[mouth_points+right_brow_points+right_eye+left_brow_points+left_eye+nose_points]

color_correct_blur_frac =0.5


cascade= cv2.CascadeClassifier("Haarcascades/haarcascade_frontalface_default.xml")
detector=dlib.get_frontal_face_detector()
predictor= dlib.shape_predictor(predictor_path)

class tooMany(Exception):
    pass
class noFaces(Exception):
    pass
def get_landmarks(im ,dlibon):
    
    if dlibon ==True:
        rects= detector(im,1)
        if len(rects) >1:
            return "error"
        if len(rects) ==0:
            return "error"
        return np.matrix([[p.x,p.y]for p in predictor(im,rects[0]).parts()])
    else:
        rects = cascade.detectMultiScale(im, 1.3,5)
        if len(rects) > 1:
            return "error"
        if len(rects) == 0:
            return "error"
        x,y,w,h =rects[0]
        rect=dlib.rectangle(x,y,x+w,y+h)
        return np.matrix([[p.x, p.y] for p in predictor(im, rect).parts()])
def annote(im,landmarks):
    im=im.copy
    for idx,points in enumerate(landmarks):
        pos =(points[0,0],points[0,1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im
#draw the convex hull
def convex_hull(im,point,color):
    points=cv2.convexHull(point)
    cv2.fillConvexPoly(im,points,color=color)
    
def get_face_mask(im,landmarks):
    im =np.zeros(im.shape[:2],dtype=np.float64)
    for p in overlay_points:
        convex_hull(im,landmarks[p],color=1)
    im=np.array([im,im,im]).transpose((1,2,0))
    
    im =(cv2.GaussianBlur(im,(feather_Amount,feather_Amount),0) > 0)*1.0
    im= cv2.GaussianBlur(im,(feather_Amount,feather_Amount),0)
    
    return im


def transformation(p1,p2):
    p1=p1.astype(np.float64)
    p2 =p2.astype(np.float64)
    
    c1=np.mean(p1,axis=0)
    c2=np.mean(p2,axis=0)
    
    p1-=c1
    p2-=c2
    
    s1 =np.std(p1)
    s2 =np.std(p2)
    
    p1/=s1
    p2/=s2
    
    u,s,vt =np.linalg.svd(p1.T*p2)
    
    r=(u*vt).T
    
    return np.vstack([np.hstack((s2/s1*r,c2.T-(s2/s1)*r*c1.T)),np.matrix([0.,0.,1.])])

# used to read the images 
def read_im_and_landmarks(im):
    im = cv2.imread(im, cv2.IMREAD_COLOR)

    im= cv2.resize(im,None,fx=1,fy=1,interpolation=cv2.INTER_LINEAR)
    im = cv2.resize(im,(im.shape[1]*scale_Factor,im.shape[0]*scale_Factor))
    s=get_landmarks(im,dlibon)
    
    return im,s


def warp_im(im,M,dshape):
    output = np.zeros(dshape,dtype= im.dtype)
    cv2.warpAffine(im,M[:2],(dshape[1],dshape[0]),dst=output,borderMode=cv2.BORDER_TRANSPARENT,flags=cv2.WARP_INVERSE_MAP)
    
    return output

def correct_colors(im1,im2,landmarks):
    blur_a =color_correct_blur_frac*np.linalg.norm(np.mean(landmarks[left_eye],axis =0)-np.mean(landmarks[right_eye],axis=0))
    blur_a=int(blur_a)
    
    if blur_a %2==0:
        blur_a+=1
    im1_blur =cv2.GaussianBlur(im1,(blur_a,blur_a),0)
    im2_blur =cv2.GaussianBlur(im2,(blur_a,blur_a),0)
    
    im2_blur+=(128*(im2 <= 1.0 )).astype(im2_blur.dtype)
    
    return (im2.astype(np.float64)* im1_blur .astype(np.float64) /im2_blur.astype(np.float64))

# Main Function
def face_swap(im1,name):
    s= get_landmarks(im1 ,True)
    
    if s=="error":
        print("no many faces")
        return im1
    
    im1,landmarks1=im1 ,s
    im2,landmarks2 = read_im_and_landmarks(name) 
    
    M=transformation(landmarks1[Align_points],landmarks2[Align_points])
    
    mask = get_face_mask(im2,landmarks2)
    
    wrapped_mask =warp_im(mask,M,im1.shape)
    
    combine_mask =np.max([get_face_mask(im1,landmarks1),wrapped_mask],axis=0)
    
    wrap_im2 =warp_im(im2,M,im1.shape)
    
    colors =correct_colors(im1, wrap_im2,landmarks1)
    
    output = im1*(1.0 -combine_mask)+colors*combine_mask
    
    cv2.imwrite('output.jpg', output)
    image = cv2.imread('output.jpg')
    
    frame = cv2.resize(image,None,fx=1.5, fy=1.5, interpolation = cv2.INTER_LINEAR)
    
    return frame
    

cap = cv2.VideoCapture("./images/Sir.mp4")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.avi', fourcc, 10.0, (640, 480),True)


filter_image =  "images/best.png" ### Put your image here!
dlibon = True
while True:   
    ret, frame = cap.read()   
    
    
    frame = cv2.flip(frame, 1)
    b=face_swap(frame, filter_image)
    
    out.write(b)
    cv2.imshow('Video Face Changer',b)
    
    
    if cv2.waitKey(1) == 13: 
        break

cap.release()
out.release()

cv2.destroyAllWindows()  